In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from linearmodels.panel import PanelOLS
import lightgbm as lgb 

# --- 1. Data Generation ---
np.random.seed(42)
n_stations = 3
n_days = 100
stations = [f'S{i+1}' for i in range(n_stations)]
dates = pd.to_datetime(pd.date_range('2024-01-01', periods=n_days, freq='D'))

data = []

station_effects = {
    'S1': {'temp_offset': 0, 'precip_factor': 1.0, 'humidity_offset': 5},
    'S2': {'temp_offset': -5, 'precip_factor': 1.5, 'humidity_offset': -5},
    'S3': {'temp_offset': 2, 'precip_factor': 0.8, 'humidity_offset': 0},
}

for station in stations:
    for date in dates:
        temp_max = 20 + station_effects[station]['temp_offset'] + 8 * np.sin(2 * np.pi * date.dayofyear / 365.25) + np.random.randn()
        sunshine = 8 - 0.2 * temp_max + np.random.randn() * 2
        precipitation = max(0, (np.random.randn() * 10 - 5) * station_effects[station]['precip_factor'])
        if precipitation > 1:
            sunshine *= 0.3
        humidity = 70 + station_effects[station]['humidity_offset'] - 1.5 * temp_max + 0.5 * precipitation + np.random.randn() * 5
        evapotranspiration = max(0, 0.1 * temp_max + 0.2 * sunshine - 0.1 * humidity/10 + np.random.randn())

        data.append({
            'date': date,
            'station_id': station,
            'temp_max': temp_max,
            'precipitation': precipitation,
            'sunshine': np.clip(sunshine, 0, 12),
            'humidity': np.clip(humidity, 10, 100),
            'evapotranspiration': evapotranspiration
        })

df = pd.DataFrame(data)

df

,date,station_id,temp_max,precipitation,sunshine,humidity,evapotranspiration
0,2024-01-01,S1,20.634327,1.476885,1.078982,52.402102,1.521055
1,2024-01-02,S1,20.041048,2.674347,2.145065,43.928230,2.536395
2,2024-01-03,S1,19.949257,0.000000,3.078689,35.509713,0.530649
3,2024-01-04,S1,19.987756,0.000000,1.976787,40.478246,0.577047
4,2024-01-05,S1,22.152898,0.000000,3.117868,34.646913,1.948011
...,...,...,...,...,...,...,...
295,2024-04-05,S3,30.622286,0.000000,1.633647,19.629111,2.755208
296,2024-04-06,S3,30.684122,9.815711,0.335253,26.883491,3.091313
297,2024-04-07,S3,30.879711,0.000000,0.000000,16.052152,3.987499
298,2024-04-08,S3,29.378290,0.000000,7.240741,26.855321,5.659534


In [7]:
# --- 2. Intra-station Modeling (Machine Learning) ---
predictors = ['temp_max', 'precipitation', 'sunshine']
targets = ['humidity', 'evapotranspiration']

X = df[predictors]
y = df[targets]

df['ml_pred_humidity'] = np.nan
df['ml_pred_evapotranspiration'] = np.nan

station_models = {}

print("\n--- شروع مدل‌سازی درون‌ایستگاهی (ML) ---")
for station in df['station_id'].unique():
    print(f"آموزش مدل برای ایستگاه: {station}")
    
    df_station = df[df['station_id'] == station]
    X_station = df_station[predictors]
    y_station = df_station[targets]
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_station, y_station)
    
    station_models[station] = model
    
    predictions = model.predict(X_station)
    
    df.loc[df['station_id'] == station, 'ml_pred_humidity'] = predictions[:, 0]
    df.loc[df['station_id'] == station, 'ml_pred_evapotranspiration'] = predictions[:, 1]

print("مدل‌سازی درون‌ایستگاهی با موفقیت انجام شد.")
print("نمونه‌ای از پیش‌بینی‌های مدل ML:")
print(df[['station_id', 'date', 'humidity', 'ml_pred_humidity', 'evapotranspiration', 'ml_pred_evapotranspiration']].head())


--- شروع مدل‌سازی درون‌ایستگاهی (ML) ---
آموزش مدل برای ایستگاه: S1
آموزش مدل برای ایستگاه: S2
آموزش مدل برای ایستگاه: S3
مدل‌سازی درون‌ایستگاهی با موفقیت انجام شد.
نمونه‌ای از پیش‌بینی‌های مدل ML:
  station_id       date   humidity  ml_pred_humidity  evapotranspiration  \
0         S1 2024-01-01  52.402102         49.492825            1.521055   
1         S1 2024-01-02  43.928230         43.236537            2.536395   
2         S1 2024-01-03  35.509713         36.953295            0.530649   
3         S1 2024-01-04  40.478246         40.316362            0.577047   
4         S1 2024-01-05  34.646913         37.222294            1.948011   

   ml_pred_evapotranspiration  
0                    1.445910  
1                    1.984991  
2                    0.830267  
3                    1.011938  
4                    2.047677  


In [8]:
# --- 3. Inter-station Modeling (Panel Data) ---
df_panel = df.set_index(['station_id', 'date'])

df['panel_pred_humidity'] = np.nan
df['panel_pred_evapotranspiration'] = np.nan

print("\n--- شروع مدل‌سازی بین‌ایستگاهی (Panel) ---")

for target_variable in targets:
    print(f"آموزش مدل پانلی برای متغیر: {target_variable}")
    
    formula = f"{target_variable} ~ 1 + {' + '.join(predictors)} + EntityEffects"
    model_panel = PanelOLS.from_formula(formula, data=df_panel)
    results = model_panel.fit()
    
    df[f'panel_pred_{target_variable}'] = results.predict().values.flatten()

print("مدل‌سازی بین‌ایستگاهی با موفقیت انجام شد.")
print("نمونه‌ای از پیش‌بینی‌های مدل Panel:")
print(df[['station_id', 'date', 'humidity', 'panel_pred_humidity', 'evapotranspiration', 'panel_pred_evapotranspiration']].head())


--- شروع مدل‌سازی بین‌ایستگاهی (Panel) ---
آموزش مدل پانلی برای متغیر: humidity
آموزش مدل پانلی برای متغیر: evapotranspiration
مدل‌سازی بین‌ایستگاهی با موفقیت انجام شد.
نمونه‌ای از پیش‌بینی‌های مدل Panel:
  station_id       date   humidity  panel_pred_humidity  evapotranspiration  \
0         S1 2024-01-01  52.402102            39.687974            1.521055   
1         S1 2024-01-02  43.928230            40.906073            2.536395   
2         S1 2024-01-03  35.509713            39.739995            0.530649   
3         S1 2024-01-04  40.478246            39.799187            0.577047   
4         S1 2024-01-05  34.646913            36.801669            1.948011   

   panel_pred_evapotranspiration  
0                       1.812417  
1                       2.021540  
2                       2.179498  
3                       1.911157  
4                       2.497807  


In [9]:
# --- 4. Ensemble Integration (Stacking with a Non-linear Meta-Learner) (بخش ویرایش شده) ---
print("\n--- شروع یکپارچه‌سازی ترکیبی (Stacking با فرا-یادگیرنده LightGBM) ---")

ensemble_models = {} # برای ذخیره مدل‌های Meta-Learner آموزش‌دیده

for target_variable in targets:
    print(f"آموزش Meta-Learner برای متغیر: {target_variable}")
    
    # متغیرهای مستقل برای مدل ترکیبی (پیش‌بینی‌های دو مدل قبلی)
    X_ensemble = df[[f'ml_pred_{target_variable}', f'panel_pred_{target_variable}']]
    
    # متغیر وابسته (مقادیر واقعی)
    y_actual = df[target_variable]
    
    # استفاده از LightGBM به عنوان یک Meta-Learner غیرخطی
    # این مدل می‌تواند روابط پیچیده بین پیش‌بینی‌های مدل‌های پایه را بیاموزد
    # و به طور هوشمند آن‌ها را ترکیب کند.
    meta_learner = lgb.LGBMRegressor(random_state=42, n_estimators=50, learning_rate=0.1)
    meta_learner.fit(X_ensemble, y_actual)
    
    # ذخیره مدل Meta-Learner
    ensemble_models[target_variable] = meta_learner
    
    # محاسبه پیش‌بینی نهایی مدل ترکیبی
    df[f'ensemble_pred_{target_variable}'] = meta_learner.predict(X_ensemble)

print("\nیکپارچه‌سازی ترکیبی با موفقیت انجام شد.")
print("نمونه‌ای از پیش‌بینی نهایی مدل Ensemble:")
print(df[['station_id', 'date', 'humidity', 'ensemble_pred_humidity', 'evapotranspiration', 'ensemble_pred_evapotranspiration']].head())



--- شروع یکپارچه‌سازی ترکیبی (Stacking با فرا-یادگیرنده LightGBM) ---
آموزش Meta-Learner برای متغیر: humidity
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000068 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 300, number of used features: 2
[LightGBM] [Info] Start training from score 34.840877
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [10]:
# --- 5. Evaluation ---
print("\n--- ارزیابی عملکرد مدل‌ها (RMSE) ---")

results_summary = {}
for target_variable in targets:
    y_true = df[target_variable]
    
    rmse_ml = np.sqrt(mean_squared_error(y_true, df[f'ml_pred_{target_variable}']))
    rmse_panel = np.sqrt(mean_squared_error(y_true, df[f'panel_pred_{target_variable}']))
    rmse_ensemble = np.sqrt(mean_squared_error(y_true, df[f'ensemble_pred_{target_variable}']))
    
    results_summary[target_variable] = {
        'ML_Only_RMSE': rmse_ml,
        'Panel_Only_RMSE': rmse_panel,
        'Ensemble_RMSE': rmse_ensemble
    }

# چاپ نتایج در قالب یک جدول
results_df = pd.DataFrame(results_summary).T
print(results_df)


--- ارزیابی عملکرد مدل‌ها (RMSE) ---
                    ML_Only_RMSE  Panel_Only_RMSE  Ensemble_RMSE
humidity                2.061479         6.264171       1.754711
evapotranspiration      0.405935         0.983929       0.288939
